In [1]:
from    typing      import  Callable

import  numpy               as      np
import  torch
import  matplotlib.pyplot   as      plt

from    tqdm.notebook       import  tqdm

from    kinetic_distribtutions      import  *

from    pathlib             import  Path
root_dir    = r"/media/junseung/47a90e46-3a9d-467c-bbee-066752b68532/GWANGJAE"
path_root   = Path(root_dir)
path_lib    = path_root / "python_deep_numerical/pytorch"
path_data   = path_root / "datasets"

from    sys         import  path
path.append( str(path_lib) )
from    torch_numerical             import  utils, distribution
from    torch_numerical.solvers     import  FastSM_Boltzmann_VHS

dtype:  torch.dtype     = torch.float64
device: torch.device    = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

dtype_and_device = {'dtype': dtype, 'device': device}
__dtype_str = str(dtype).split('.')[-1]

In [2]:
DELTA_T:    float   = 0.1
MAX_T:      float   = 5.0
NUM_INST:   int     = 20
DATA_SIZE:  int     = NUM_INST * int(MAX_T/DELTA_T + 0.1)

T1__n_init  = T2__n_init    = T3__n_init    = NUM_INST
T1__size    = T2__size      = T3__size      = DATA_SIZE

DIMENSION:  int     = 2
RESOLUTION: int     = (2**6)+1
V_MAX:      float   = 3.0/utils.LAMBDA
DELTA_V:    float   = (2*V_MAX) / RESOLUTION
V_WHERE_CLOSED: str = 'both' if RESOLUTION%2==1 else 'none'

v_grid = utils.velocity_grid(DIMENSION, RESOLUTION, V_MAX, where_closed=V_WHERE_CLOSED, **dtype_and_device)

FFT_AXES:   tuple[int] = tuple(range(-(1+DIMENSION), -1))
FFT_NORM:   str  = 'forward'

VHS_COEFF = 1 / utils.area_of_unit_sphere(DIMENSION)
VHS_ALPHA = -2

sample_q: Callable[[int], tuple[torch.Tensor]] = \
    lambda batch_size: sample_quantities(DIMENSION, batch_size, **dtype_and_device)

# Saved information
dataset_info: dict[str, object] = {
    'dimension':    DIMENSION,
    'resolution':   RESOLUTION,
    'v_max':        V_MAX,
    'vhs_coeff':    VHS_COEFF,
    'equation':     'Boltzmann',
}

In [3]:
solver = FastSM_Boltzmann_VHS(
    dimension   = DIMENSION,
    v_num_grid  = RESOLUTION,
    v_max       = V_MAX,
    
    vhs_coeff   = VHS_COEFF,
    vhs_alpha   = VHS_ALPHA,
    
    quad_order_uniform  = 25,
    quad_order_legendre = 25,
    quad_order_lebedev  = 7,
    
    **dtype_and_device,
)
FFT_CONFIG: dict[str, object] = {'s': solver.v_shape, 'dim': solver.v_axes, 'norm': FFT_NORM}

Type 1. Maxwellian distribution

In [4]:
T1__data:       list[torch.Tensor]  = []
T1__collision:  list[torch.Tensor]  = []

T1__init: torch.Tensor = distribution.maxwellian_homogeneous(v_grid, *sample_q(T1__n_init))
arr_f_1 = T1__init = normalize_density(T1__init, DELTA_V)
arr_f_1_fft: torch.Tensor = torch.fft.fftn(arr_f_1, **FFT_CONFIG)

for cnt in tqdm(range(T1__size//T1__n_init)):
    ##### 1. Save the distribution at the previous time step
    T1__data.append(arr_f_1)
    ##### 2. Save the collision term at the previous time step
    _col_1_fft = solver.compute_fft(None, arr_f_1_fft)
    col_1 = torch.real(torch.fft.ifftn(_col_1_fft, **FFT_CONFIG))
    T1__collision.append(col_1)
    ##### 3. Compute the distribution at the current time step
    arr_f_1_fft = solver.forward(0.0, arr_f_1_fft, DELTA_T, utils.one_step_RK4_classic)
    arr_f_1 = torch.real(torch.fft.ifftn(arr_f_1_fft, **FFT_CONFIG))
    
T1__data:       torch.Tensor    = torch.concatenate(T1__data, dim=0).cpu()
T1__collision:  torch.Tensor    = torch.concatenate(T1__collision, dim=0).cpu()


print(f"The shape of the input data")
print(f">>> {T1__data.shape}")
print(f"The shape of the output data")
print(f">>> {T1__collision.shape}")

  0%|          | 0/50 [00:00<?, ?it/s]

The shape of the input data
>>> torch.Size([1000, 1, 1, 65, 65, 1])
The shape of the output data
>>> torch.Size([1000, 1, 1, 65, 65, 1])


Type 2. Sum of two Maxwellian distributions

In [5]:
T2__data:       list[torch.Tensor]  = []
T2__collision:  list[torch.Tensor]  = []

T2__init: torch.Tensor = \
    0.5 * (
        distribution.maxwellian_homogeneous(v_grid, *sample_q(T2__n_init))
        +
        distribution.maxwellian_homogeneous(v_grid, *sample_q(T2__n_init))
    )
arr_f_2 = T2__init = normalize_density(T2__init, DELTA_V)
arr_f_2_fft: torch.Tensor = torch.fft.fftn(arr_f_2, **FFT_CONFIG)

for cnt in tqdm(range(T2__size//T2__n_init)):
    ##### 1. Save the distribution at the previous time step
    T2__data.append(arr_f_2)
    ##### 2. Save the collision term at the previous time step
    _col_2_fft = solver.compute_fft(None, arr_f_2_fft)
    col_2 = torch.real(torch.fft.ifftn(_col_2_fft, **FFT_CONFIG))
    T2__collision.append(col_2)
    ##### 3. Compute the distribution at the current time step
    arr_f_2_fft = solver.forward(0.0, arr_f_2_fft, DELTA_T, utils.one_step_RK4_classic)
    arr_f_2 = torch.real(torch.fft.ifftn(arr_f_2_fft, **FFT_CONFIG))
    
T2__data:       torch.Tensor    = torch.concatenate(T2__data, dim=0).cpu()
T2__collision:  torch.Tensor    = torch.concatenate(T2__collision, dim=0).cpu()

print(f"The shape of the input data")
print(f">>> {T2__data.shape}")
print(f"The shape of the output data")
print(f">>> {T2__collision.shape}")

  0%|          | 0/50 [00:00<?, ?it/s]

The shape of the input data
>>> torch.Size([1000, 1, 1, 65, 65, 1])
The shape of the output data
>>> torch.Size([1000, 1, 1, 65, 65, 1])


Type 3. Perturbed Maxwellian distributions

In [6]:
T3__data:       list[torch.Tensor] = []
T3__collision:  list[torch.Tensor] = []

coeffs = sample_noise_quadratic(DIMENSION, V_MAX, T3__n_init, **dtype_and_device)
quad = compute_quadratic_polynomial(v_grid, coeffs)
quad = quad.reshape(T3__n_init, *utils.ones(DIMENSION), *utils.repeat(RESOLUTION, DIMENSION), 1)

T3__init: torch.Tensor = \
    distribution.maxwellian_homogeneous(v_grid, *sample_q(T3__n_init)) * \
    (1 + quad)
arr_f_3 = T3__init = normalize_density(T3__init, DELTA_V)
arr_f_3_fft: torch.Tensor = torch.fft.fftn(arr_f_3, **FFT_CONFIG)

for cnt in tqdm(range(T3__size//T3__n_init)):
    ##### 1. Save the distribution at the previous time step
    T3__data.append(arr_f_3)
    ##### 2. Save the collision term at the previous time step
    _col_3_fft = solver.compute_fft(None, arr_f_3_fft)
    col_3 = torch.real(torch.fft.ifftn(_col_3_fft, **FFT_CONFIG))
    T3__collision.append(col_3)
    ##### 3. Compute the distribution at the current time step
    arr_f_3_fft = solver.forward(0.0, arr_f_3_fft, DELTA_T, utils.one_step_RK4_classic)
    arr_f_3 = torch.real(torch.fft.ifftn(arr_f_3_fft, **FFT_CONFIG))
    
T3__data:       torch.Tensor    = torch.concatenate(T3__data, dim=0).cpu()
T3__collision:  torch.Tensor    = torch.concatenate(T3__collision, dim=0).cpu()

print(f"The shape of the input data")
print(f">>> {T3__data.shape}")
print(f"The shape of the output data")
print(f">>> {T3__collision.shape}")

  0%|          | 0/50 [00:00<?, ?it/s]

The shape of the input data
>>> torch.Size([1000, 1, 1, 65, 65, 1])
The shape of the output data
>>> torch.Size([1000, 1, 1, 65, 65, 1])


Merge the data

In [7]:
data        = \
    torch.concatenate((T1__data, T2__data, T3__data), dim=0)
collision   = \
    torch.concatenate((T1__collision, T2__collision, T3__collision), dim=0)

In [8]:
saved_data: dict[str, object] = {
    'input_distribution':   data,
    'collision_term':       collision,
    
    'max_t':            MAX_T,
    'delta_t':          DELTA_T,
    
    'resolution':       RESOLUTION,
    'max_v':            V_MAX,
    'v_where_closed':   V_WHERE_CLOSED,
    
    'vhs_coeff':    VHS_COEFF,
    'vhs_alpha':    VHS_ALPHA,
    
    'equation':     'Landau',
    'dtype_str':    __dtype_str,
}
file_dir = path_data / __dtype_str
file_name = f"Boltzmann__{DIMENSION}D__res_{str(RESOLUTION).zfill(3)}__alpha_{float(VHS_ALPHA):.1e}.pth"
if not Path.exists(file_dir):
    Path.mkdir(file_dir, parents=True)
torch.save(saved_data, file_dir/file_name)

End of file